In [1]:
from sionna.rt import load_scene
import mitsuba as mi

import drjit as dr
import numpy as np
import torch

# scene = load_scene(filename="cubes/cubes.xml")
# scene = load_scene(filename="cubes/hugging_cubes.xml")
scene = load_scene(filename="minis2/row4_col4.xml")

mi_scene = scene.mi_scene

In [2]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)
print(shapes)

shape_ptrs = dr.reinterpret_array(mi.UInt32, shapes).numpy()
print(shape_ptrs)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []

# shape_ptr_to_offset = np.zeros(shape=shape_ptrs.shape[0]+1, dtype=int)
shape_ptr_to_offset = {}
offset_counter = 0
for i, s in enumerate(shapes):
    shape_ptr_to_offset[shape_ptrs[i]] = offset_counter
    offset_counter += s.primitive_count()

    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
# print(face_averages)
# print(face_normals)

print(shape_ptr_to_offset)


[Mesh[
  name = "Church_Street_Surplus-wall.ply + The_First_National_City_Bank_of_New_York-wall.ply + element_3354-wall.ply + element_3369-wall.ply + element_3371-wall.ply + element_3372-wall.ply + element_3375-wall.ply + element_3433-wall.ply + element_3434-wall.ply + element_3444-wall.ply + element_3452-wall.ply + element_3454-wall.ply + element_3466-wall.ply + element_5568-wall.ply + element_5569-wall.ply + element_5570-wall.ply + element_5571-wall.ply + element_5572-wall.ply + element_5573-wall.ply + element_5574-wall.ply + element_5575-wall.ply + element_5576-wall.ply + element_5577-wall.ply + element_5578-wall.ply + element_5582-wall.ply + element_5583-wall.ply + element_5585-wall.ply + element_5588-wall.ply + element_5589-wall.ply + element_5590-wall.ply + element_5592-wall.ply + element_5593-wall.ply + element_5594-wall.ply + element_5595-wall.ply + element_5596-wall.ply + element_5597-wall.ply + element_5599-wall.ply + element_5601-wall.ply + element_5602-wall.ply + element_56

In [3]:
# Generate rays to be shot from each surface in 180 cone
# Currently generates 360 then filters based on dot product

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
ray_per_source = 10000

sources = face_averages + (face_normals * 0.0001)

sky_test = mi.Ray3f(sources, mi.Vector3f([0,0,1]))
prelim_int = mi_scene.ray_intersect_preliminary(sky_test)
exterior_source = ~prelim_int.is_valid()
exterior_source


[True, True, True, .. 1163 skipped .., True, True, True]

In [4]:

rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
# print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
exterior_source = dr.repeat(exterior_source, ray_per_source)
# print(norms_expanded)
# shouldn't this be positive dot prod???
print(rays.d)
active = exterior_source & (dr.dot(rays.d, norms_expanded) > 0)
# print(active)

[[0, 0, 1],
 [-0.0147479, -0.0135103, 0.9998],
 [0.00247273, 0.0281755, 0.9996],
 .. 11689994 skipped ..,
 [-0.0281711, 0.00252272, -0.9996],
 [0.0158941, 0.012141, -0.9998],
 [-0, -0, -1]]


In [5]:
import numpy as np

valid = np.array([True, True, False, True])
print(valid)
indices = np.nonzero(valid)[0]
print(indices)
add_mask = np.array([False, True, True])
print(add_mask)
set_false = indices[~add_mask]
valid[set_false] = False
print(valid)

[ True  True False  True]
[0 1 3]
[False  True  True]
[False  True False  True]


In [6]:
interactions = mi_scene.ray_intersect_preliminary(rays, active=active)
# print(interactions)
full_int = interactions.compute_surface_interaction(rays, active=active)
# print(full_int.p)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

valid = full_int.is_valid().numpy()
interaction_normals = full_int.n.numpy().T[valid]

# make sure the dot product is negative
# outgoing rays dirs
# maybe keep this in dr.jit
ray_dir_np = rays.d.numpy().T[valid]
print(ray_dir_np)
ray_dir_dot_interaction_normal_np = np.sum(ray_dir_np * interaction_normals, axis=1)
valid_hits_exterior = ray_dir_dot_interaction_normal_np < 0

indices = np.nonzero(valid)[0]
print(indices)

invalid = indices[~valid_hits_exterior]
valid[invalid] = False

# a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
# a=dr.select(full_int.is_valid(), full_int.prim_index, dr.cuda.UInt(count)).numpy()
# interaction_shapes=dr.select(full_int.is_valid(), full_int.shape, dr.cuda.UInt32(36000)).numpy()

indices = np.nonzero(valid)[0]
interaction_shapes = full_int.shape.numpy()[valid]
prim_indices = full_int.prim_index.numpy()[valid]
# interaction_shapes[0].id()
# print(interaction_normals)

[[ 2.47272756e-03  2.81754900e-02  9.99599934e-01]
 [ 3.77264507e-02  2.39984822e-02  9.98999894e-01]
 [-2.43813507e-02  4.69448119e-02  9.98599887e-01]
 ...
 [-4.01996404e-01  9.15641129e-01  5.00023365e-04]
 [ 9.14926529e-01 -4.03620332e-01  3.00049782e-04]
 [-9.47280169e-01 -3.20406526e-01  1.00016594e-04]]
[       2        5        7 ... 11684997 11684998 11684999]


In [7]:
# print("shapes:", mi_scene.shapes())
# print(full_int.shape_index)
# # wtf=dr.reinterpret_array(mi.UInt32, full_int.shape)
# # wtf2 = dr.reinterpret_array(mi.ShapePtr, wtf)
# # # print(wtf)
# # for w in wtf2:
# #     print(w.id())
# print(wtf2[3598].has_mesh_attributes())
# np.unique(wtf)

In [8]:
# full_valid = full_int.is_valid().numpy()
# print(full_valid.shape)
# hit_points = full_int.p.numpy().T[full_valid]

# origins = rays.o.numpy().T[full_valid]
# print(origins.shape)

# scene.preview()

# # This cell is lowkey pretty illegal but here's to being easier to implement
# ## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)


# colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
# colors = np.tile(colors, (origins.shape[0], 1))
# # print(colors)
# # print(colors.dtype)
# width = np.array(1.0)
# scene._preview_widget._plot_lines(origins, hit_points, colors, width)

In [9]:
# dir(full_int.prim_index)
# collision_shape_i = full_int.prim_index.numpy()[full_valid]

# np.unique(collision_shape_i)

In [10]:
scene.preview()
None

In [11]:
sets = [set() for _ in range(count)]

for i, shape_i, prim_i in zip(indices, interaction_shapes, prim_indices):
    # print(i, shape_i, prim_i)
    # shape_i -= 1
    # shape_id = mi_scene.shapes()[shape_i].id()
    offset = shape_ptr_to_offset[shape_i]
    sets[i // ray_per_source].add(offset + prim_i)

# for i in range(len(a)):
#     vis = a[i]
#     if vis < count:
#         cur_shape = interaction_shapes[i] - 1
#         cur_shape = mi_scene.shapes()[cur_shape].id()
#         offset = shape_offset[cur_shape]
#         # offset = mi_scene.shapes[collision_shape_i].id()
#         sets[i // ray_per_source].add(vis + offset)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

[155.4336    230.62729     1.6666667] {720, 745, 744}
[161.77779   226.54124     3.3333335] {744}
[146.78857   230.88419     1.6666667] {692}
[147.93898   232.79875     3.3333335] {692}
[158.32655   220.79753     1.6666667] set()
[151.98236   224.88358     3.3333335] set()
[166.97157   220.54063     1.6666667] {822}
[165.82117   218.62607     3.3333335] {822}
[273.21423   135.8097      5.4000006] {837, 686, 687, 688, 689, 337, 336, 338, 339, 374, 375, 376, 377, 94}
[271.42535  133.0083    10.800001] {837, 686, 687, 688, 689, 337, 339, 336, 338, 374, 375, 376, 377, 94}
[275.0144    139.07121     5.4000006] {34, 35, 40, 41, 687, 336, 337, 374, 375, 376, 377, 126, 87, 127, 86, 94, 95}
[275.0088   138.84116   10.800001] {34, 921, 35, 40, 41, 922, 687, 336, 337, 374, 375, 376, 377, 87, 86, 127, 94, 95}
[256.04382   157.10654     5.4000006] {132, 133, 40, 41, 42, 43, 56, 57, 68, 69, 84, 85, 92, 93, 94, 95, 108, 109, 116, 117, 124, 125, 127}
[265.53195  148.2039    10.800001] {132, 133, 921, 

In [12]:
face_averages


[[155.434, 230.627, 1.66667],
 [161.778, 226.541, 3.33333],
 [146.789, 230.884, 1.66667],
 .. 1163 skipped ..,
 [212.547, 172.491, 16.1],
 [216.74, 177.066, 16.1],
 [212.963, 176.046, 16.1]]

In [13]:
sizes = [len(s) for s in sets]
face_averages_np = face_averages.numpy().T
starts = np.repeat(face_averages_np, repeats=sizes, axis=0)
print(starts.shape)

ends_indices = []
for s in sets:
    for shape in s:
        ends_indices.append(shape)

ends_indices = np.array(ends_indices)
print(ends_indices)
ends = face_averages_np[ends_indices]
ends.shape

print(ends)

(20170, 3)
[720 745 744 ... 888 889 765]
[[153.66756   231.76663     1.8000001]
 [160.01175   227.68059    17.333334 ]
 [164.06686   225.0679      8.666667 ]
 ...
 [210.85703   179.51163    11.2      ]
 [208.84587   176.1685     22.4      ]
 [189.0907    131.77612    15.800001 ]]


In [14]:
scene.preview()

# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
colors = np.array([0.0, 1.0, 0.0, 1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)